In [2]:
import numpy as np
import pdb
from walk import RWGraph
import networkx as nx
from collections import defaultdict
from gensim.models.keyedvectors import Vocab
from numpy import random
from six import iteritems
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.optim as optim
from sklearn.metrics import (auc, f1_score, precision_recall_curve, roc_auc_score)

In [3]:
#路径设定
file_name = '../Tdata'
#超参数设定
embedding_size = 200
embedding_u_size = 10
num_sampled = 5 #负采样个数
dim_a = 20 #attention维度
att_head = 1
neighbor_samples = 5
schema = None # metapath的制定schema
num_walks = 20 #路径数
walk_length = 10 #
window_size = 5 #上下文窗口大小,应小于walk_length

In [4]:
#数据读取
def load_training_data(f_name):
    print('We are loading data from:', f_name)
    edge_data_by_type = dict()
    all_edges = list()
    all_nodes = list()
    with open(f_name, 'r') as f:
        for line in f:
            words = line[:-1].split(' ')
            if words[0] not in edge_data_by_type:
                edge_data_by_type[words[0]] = list()
            x, y = words[1], words[2]
            edge_data_by_type[words[0]].append((x, y))
            all_edges.append((x, y))
            all_nodes.append(x)
            all_nodes.append(y)
    all_nodes = list(set(all_nodes))
    all_edges = list(set(all_edges))
    edge_data_by_type['Base'] = all_edges
    print('Total training nodes: ' + str(len(all_nodes)))
    return edge_data_by_type
def load_testing_data(f_name):
    print('We are loading data from:', f_name)
    true_edge_data_by_type = dict()
    false_edge_data_by_type = dict()
    all_edges = list()
    all_nodes = list()
    with open(f_name, 'r') as f:
        for line in f:
            words = line[:-1].split(' ')
            x, y = words[1], words[2]
            if int(words[3]) == 1:
                if words[0] not in true_edge_data_by_type:
                    true_edge_data_by_type[words[0]] = list()
                true_edge_data_by_type[words[0]].append((x, y))
            else:
                if words[0] not in false_edge_data_by_type:
                    false_edge_data_by_type[words[0]] = list()
                false_edge_data_by_type[words[0]].append((x, y))
            all_nodes.append(x)
            all_nodes.append(y)
    all_nodes = list(set(all_nodes))
    return true_edge_data_by_type, false_edge_data_by_type

In [5]:
network_data = load_training_data(file_name + '/train.txt')
valid_true_data_by_edge, valid_false_data_by_edge = load_testing_data(file_name + '/valid.txt')
testing_true_data_by_edge, testing_false_data_by_edge = load_testing_data(file_name + '/test.txt')

We are loading data from: ../Tdata/train.txt
Total training nodes: 511
We are loading data from: ../Tdata/valid.txt
We are loading data from: ../Tdata/test.txt


In [6]:
#路径生成
#从edgse生成Graph
def get_G_from_edges(edges):
    edge_dict = dict()
    for edge in edges:
        edge_key = str(edge[0]) + '_' + str(edge[1])
        if edge_key not in edge_dict:
            edge_dict[edge_key] = 1
        else:
            edge_dict[edge_key] += 1
    tmp_G = nx.Graph()
    for edge_key in edge_dict:
        weight = edge_dict[edge_key]
        x = edge_key.split('_')[0]
        y = edge_key.split('_')[1]
        tmp_G.add_edge(x, y)
        tmp_G[x][y]['weight'] = weight
    return tmp_G

def generate_walks(network_data):
    base_network = network_data['Base']
    if schema is not None:
        node_type = load_node_type('../data/node_type.txt')
    else:
        node_type = None
        
    base_walker = RWGraph(get_G_from_edges(base_network), node_type=node_type)
    base_walks = base_walker.simulate_walks(num_walks, walk_length, schema=schema)
    all_walks = []
    for layer_id in network_data:
        if layer_id == 'Base':
            continue

        tmp_data = network_data[layer_id]
        # start to do the random walk on a layer

        layer_walker = RWGraph(get_G_from_edges(tmp_data))
        layer_walks = layer_walker.simulate_walks(num_walks, walk_length)

        all_walks.append(layer_walks)

    print('路径生成完毕！')

    return base_walks, all_walks

In [7]:
base_walks, all_walks = generate_walks(network_data)

路径生成完毕！


In [8]:
#生成index2word
def generate_vocab(all_walks):
    index2word = []
    raw_vocab = defaultdict(int)

    for walks in all_walks:
        for walk in walks:
            for word in walk:
                raw_vocab[word] += 1

    vocab = {}
    for word, v in iteritems(raw_vocab):
        vocab[word] = Vocab(count=v, index=len(index2word))
        index2word.append(word)

    index2word.sort(key=lambda word: vocab[word].count, reverse=True)
    for i, word in enumerate(index2word):
        vocab[word].index = i
    
    return vocab, index2word

In [9]:
vocab, index2word = generate_vocab([base_walks])

In [10]:
with open(file_name+'/feature.txt', 'r') as f:
    first = True
    for line in f:
        if first:
            items_first = line.strip().split() 
            feature_dic = np.zeros([len(vocab),int(items_first[1])])            
            first = False
            continue
        items = line.strip().split()
        if items[0] in vocab:
            feature_dic[vocab[items[0]].index] = np.array(items[1:],dtype=float)
feature_dic = torch.Tensor(feature_dic)

In [11]:
def generate_pairs(all_walks, vocab):
    pairs = []
    skip_window = window_size // 2
    for layer_id, walks in enumerate(all_walks):
        for walk in walks:
            for i in range(len(walk)):
                for j in range(1, skip_window + 1):
                    if i - j >= 0:
                        pairs.append((vocab[walk[i]].index, vocab[walk[i - j]].index, layer_id))
                    if i + j < len(walk):
                        pairs.append((vocab[walk[i]].index, vocab[walk[i + j]].index, layer_id))
    return pairs

In [12]:
train_pairs = generate_pairs(all_walks, vocab)

In [13]:
# 确保Base边在最后一个
edge_types = list(network_data.keys())
if edge_types[-1] != 'Base':
    edge_types.sort()
    edge_types.remove('Base')
    edge_types.append('Base')

In [14]:
# 网络参数计算
num_nodes = len(index2word)
edge_type_count = len(edge_types) - 1
u_num = edge_type_count
neighbors = [[[] for __ in range(edge_type_count)] for _ in range(num_nodes)]

In [15]:
#根据neighbor_samples去截断、填充邻居，如果一个实体点没有邻居，则认为它的邻居是他自己
for r in range(edge_type_count):
    g = network_data[edge_types[r]]
    for (x, y) in g:
        ix = vocab[x].index
        iy = vocab[y].index
        neighbors[ix][r].append(iy)
        neighbors[iy][r].append(ix)
    for i in range(num_nodes):
        if len(neighbors[i][r]) == 0:
            neighbors[i][r] = [i] * neighbor_samples
        elif len(neighbors[i][r]) < neighbor_samples:
            neighbors[i][r].extend(list(np.random.choice(neighbors[i][r], size=neighbor_samples-len(neighbors[i][r]))))
        elif len(neighbors[i][r]) > neighbor_samples:
            neighbors[i][r] = list(np.random.choice(neighbors[i][r], size=neighbor_samples))

In [16]:
# 进行负采样
def negative_sampling(targets, vocab, k):
    batch_size = targets.size(0)
    neg_samples = []
    for i in range(batch_size):
        nsample = []
        target_index = targets[i].item()
        while len(nsample) < k: # num of sampling
            neg = random.choice(len(vocab))
            if neg == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(np.array(nsample)) 
    return np.array(neg_samples)

In [17]:
def evaluate(model, true_edges, false_edges):

    true_list = list()
    prediction_list = list()
    true_num = 0
    for edge in true_edges:
        tmp_score = get_score(model, str(edge[0]), str(edge[1]))
        
        if tmp_score is not None:
            true_list.append(1)
            prediction_list.append(tmp_score)
            true_num += 1
    for edge in false_edges:
        tmp_score = get_score(model, str(edge[0]), str(edge[1]))
        
        if tmp_score is not None:
            true_list.append(0)
            prediction_list.append(tmp_score)

    sorted_pred = prediction_list[:]
    sorted_pred.sort()
    threshold = sorted_pred[-true_num]
    y_pred = np.zeros(len(prediction_list), dtype=np.int32)
    for i in range(len(prediction_list)):
        if prediction_list[i] >= threshold:
            y_pred[i] = 1
    y_true = np.array(true_list)
    y_scores = np.array(prediction_list)
    ps, rs, _ = precision_recall_curve(y_true, y_scores)
    return roc_auc_score(y_true, y_scores), f1_score(y_true, y_pred), auc(rs, ps)

In [18]:
def get_score(local_model, node1, node2):

    try:
        vector1 = np.array(local_model[node1].view(-1).tolist())
        vector2 = np.array(local_model[node2].view(-1).tolist())
        return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    except Exception as e:
        pass

In [19]:
def getBatch(pairs, neighbors, batch_size):
    n_batches = (len(pairs) + (batch_size - 1)) // batch_size
    for idx in range(n_batches):
        x, y, t, neigh = [], [], [], []
        for i in range(batch_size):
            index = idx * batch_size + i
            if index >= len(pairs):
                break
            x.append(pairs[index][0])
            y.append(pairs[index][1])
            t.append(pairs[index][2])
            neigh.append(neighbors[pairs[index][0]])
        yield (np.array(x).astype(np.int32), np.array(y).reshape(-1, 1).astype(np.int32), np.array(t).astype(np.int32), np.array(neigh).astype(np.int32)) 

In [20]:
# 网络结构,核心代码
class MyLayer(nn.Module):
    def __init__(self, num_nodes, embedding_size, u_num, dim_a, att_head, edge_type_count, feature_dic):
        super(MyLayer, self).__init__()
        # attention参数传递
        self.u_num = u_num
        self.att_head = att_head
        self.embedding_size = embedding_size
        feature_dim = feature_dic.shape[1]
        self.edge_type_count = edge_type_count
        self.embedding_u_size = embedding_u_size
        # 初始化层
        self.feature_dic = feature_dic # Tensor(node_num*feature_dim)
        self.feature_layer1 = nn.Linear(feature_dim, embedding_size)
        self.feature_layer2 = nn.Linear(feature_dim, embedding_size)
        self.neigh_feature_trans = nn.Parameter(torch.Tensor(edge_type_count, feature_dim, embedding_u_size))
        self.neigh_linear_1 = nn.Linear(embedding_u_size, dim_a)
        self.neigh_linear_2 = nn.Linear(dim_a, att_head)
        self.neigh_linear_last = nn.Linear(embedding_u_size, embedding_size // att_head)
    def forward(self,input_node,node_neigh):
        # input_node:LongTensor(1),label_node:LongTensor(1)
        # node_neigh:LongTensor(edge_type_num*neigh_sample)
        batch_size = max(1,input_node.shape[0])
        node_feature = torch.index_select(self.feature_dic, 0, input_node) # 提取输入node的特征
        node_embed = self.feature_layer1(node_feature)
        node_weight = self.feature_layer2 (node_feature)
        node_neigh = torch.unbind(node_neigh, dim=1)
        neigh_type_embedding = torch.empty([batch_size,self.edge_type_count,self.embedding_u_size]).cuda()
        for j in range(batch_size):
            neigh_feature_temp = torch.cat([torch.matmul(torch.index_select(self.feature_dic, 0, node_neigh[i][j]), self.neigh_feature_trans[i]) for i in range(self.edge_type_count)])
            neigh_type_embedding[j] = torch.mean(neigh_feature_temp,0)
        # attention层
        attention = nn.Tanh()(self.neigh_linear_1(neigh_type_embedding))
        attention = self.neigh_linear_2(attention)
        attention = nn.Softmax()(attention.view(-1,self.u_num))
        attention = attention.view(-1, self.att_head, self.u_num)
        node_type_embed = torch.matmul(attention,neigh_type_embedding)
        node_embed = node_embed + self.neigh_linear_last(node_type_embed).view(-1,self.embedding_size) + node_weight
        last_node_embed = nn.functional.normalize(node_embed, p=2, dim=1, eps=1e-12, out=None)
        return last_node_embed.reshape(batch_size,1,-1)

In [21]:
class MyNet(nn.Module):
    def __init__(self,num_nodes, embedding_size, u_num, dim_a, att_head, edge_type_count, feature_dic):
        super(MyNet, self).__init__()
        self.Embedding_layer = MyLayer(num_nodes, embedding_size, u_num, dim_a, att_head, edge_type_count, feature_dic)
        self.embedding_u = nn.Embedding(num_nodes, embedding_size)
        self.logsigmoid = nn.LogSigmoid()
        # 初始化out_embedding
        self.embedding_u.weight.data.uniform_(-0.0, 0.0)
    def forward(self, input_node, node_neigh, label_node, neg_node):
        center_embeds = self.Embedding_layer(input_node,node_neigh)
        target_embeds = self.embedding_u(label_node)
        
        neg_embeds = -self.embedding_u(neg_node)
        
        positive_score = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # Bx1
        negative_score = torch.sum(neg_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2), 1).view(neg_node.size(0), -1) # BxK -> Bx1
        
        loss = self.logsigmoid(positive_score) + self.logsigmoid(negative_score)
        
        return -torch.mean(loss)
    def prediction(self,input_node,node_neigh):
        return self.Embedding_layer(input_node,node_neigh)

In [22]:
# 模型训练参数
BATCH_SIZE = 256
EPOCH = 200
NEG = 5
set_patience = 4

In [23]:
losses = []
model = MyNet(num_nodes, embedding_size, u_num, dim_a, att_head, edge_type_count, feature_dic.cuda()).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)
g_iter = 0
best_score = 0
patience = 5

In [24]:
for epoch in range(EPOCH):
    for i,batch in enumerate(getBatch(train_pairs,neighbors,BATCH_SIZE)):

        input_node = torch.LongTensor(batch[0]).cuda()
        label_node = torch.LongTensor(batch[1]).cuda()
        node_neigh = torch.LongTensor(batch[3]).cuda()
        neg_node = torch.LongTensor(negative_sampling(label_node, vocab, NEG)).cuda()
        model.zero_grad()

        loss = model(input_node, node_neigh, label_node, neg_node)
        
        loss.backward()
        optimizer.step()   
        losses.append(loss.data.tolist())
    if epoch % 10 == 0:
        print("Epoch : %d, mean_loss : %.02f" % (epoch, np.mean(losses)))
        losses = []
    final_model = dict(zip(edge_types[:-1], [dict() for _ in range(edge_type_count)]))
    for i in range(edge_type_count):
        for j in range(num_nodes):
            input_node = torch.LongTensor([j]).cuda()
            node_neigh = torch.LongTensor(neighbors[j]).cuda()
            final_model[edge_types[i]][index2word[j]] = model.prediction(input_node,node_neigh)
            valid_aucs, valid_f1s, valid_prs = [], [], []
            test_aucs, test_f1s, test_prs = [], [], []
    for i in range(edge_type_count):
        tmp_auc, tmp_f1, tmp_pr = evaluate(final_model[edge_types[i]], valid_true_data_by_edge[edge_types[i]], valid_false_data_by_edge[edge_types[i]])
        valid_aucs.append(tmp_auc)
        valid_f1s.append(tmp_f1)
        valid_prs.append(tmp_pr)

        tmp_auc, tmp_f1, tmp_pr = evaluate(final_model[edge_types[i]], testing_true_data_by_edge[edge_types[i]], testing_false_data_by_edge[edge_types[i]])
        test_aucs.append(tmp_auc)
        test_f1s.append(tmp_f1)
        test_prs.append(tmp_pr)
    print('valid auc:', np.mean(valid_aucs))
    print('valid pr:', np.mean(valid_prs))
    print('valid f1:', np.mean(valid_f1s))

    average_auc = np.mean(test_aucs)
    average_f1 = np.mean(test_f1s)
    average_pr = np.mean(test_prs)

    cur_score = np.mean(valid_aucs)
    if cur_score > best_score:
        best_score = cur_score
        patience = 0
    else:
        patience += 1
        if patience > set_patience:            
            output_name = '../Tdata/result_pytorch.txt'
            f = open(output_name,'w+')
            f.write('Overall ROC-AUC:' + str(average_auc) + '\n')
            f.write('Overall PR-AUC'+ str(average_pr) + '\n')
            f.write('Overall F1:'+ str(average_f1) + '\n') 
            print('Early Stopping')
            break    

/root/miniconda3/envs/wld_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch : 0, mean_loss : 1.02
valid auc: 0.7605699985380243
valid pr: 0.6912321270006594
valid f1: 0.6747067448680352


/root/miniconda3/envs/wld_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


valid auc: 0.7629230054781091
valid pr: 0.6798681835336058
valid f1: 0.7005131964809383


/root/miniconda3/envs/wld_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

Epoch : 10, mean_loss : 0.61
valid auc: 0.8119275827521263
valid pr: 0.7638558736939722
valid f1: 0.729692082111437


/root/miniconda3/envs/wld_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


valid auc: 0.8075977588772028
valid pr: 0.7629593023920498
valid f1: 0.729692082111437


/root/miniconda3/envs/wld_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


valid auc: 0.8112711341491732
valid pr: 0.7663238331874105
valid f1: 0.729692082111437
Early Stopping
